## Getting Post Codes for Canada

In [1]:
import pandas as pd
import numpy as np

Read Tables into List  
Read selected table into dataframe  
Drop rows with Borough not assigned

In [2]:
pagename = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
mylist =(pd.read_html(pagename, header = 0))
df = pd.DataFrame(mylist[0],columns =['Postcode','Borough','Neighbourhood'])
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace = True)
df = df[df['Borough']!= "Not assigned"]

df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Group Postcodes and aggerate Neighbour columns  
Assigh Bourgh to all neighgbours that are not assigned

In [3]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].agg(', '.join).reset_index()
df.loc[df['Neighborhood'] == 'Not assigned'] = df['Borough']

In [4]:
df.shape

(103, 3)

## Get Lat and Long for each postcode

Tried the geocoder but just hangs without any output
Moved to using csv file

In [5]:
filename = 'http://cocl.us/Geospatial_data'
ll_df = pd.read_csv(filename)

Rename column in lat and long dataframe to match original dataframe
Create new dataframe with merged data from both dataframe using postcode as common column

In [6]:
ll_df.rename(columns={'Postal Code':'Postcode'}, inplace = True)
df1 = pd.merge(df,ll_df, on ='Postcode')
df1.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Explore Tortonto Neighbor

Install modules

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

In [92]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

# All requested packages already installed.



In [7]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Look at Bough Counts

In [213]:
df1['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

Filter for choosen Borough

In [343]:
df2 = df1[df1['Borough'].str.contains('Downtown Toronto')]
df2

,Postcode,Borough,Neighborhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [344]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of all Neighboods in Borough

In [345]:
# create map of Torontoa using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [346]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Test API

In [347]:
# test
search_query = 'Italian'
radius = 500
LIMIT = 100
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Italian .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=FZ3YCJJ5SFRN4EMRJRWTY4YQZOGZFOJ35GVE55V1YEAVGU3Q&client_secret=3ZKXP3XO3QJWQGAMEMZ1O4EYOIVZ5ZEAEXWGB0FB12TOIPA2&ll=43.653963,-79.387207&v=20180605&query=Italian&radius=500&limit=100'

In [348]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d76c27da306190035d6ce24'},
 'response': {'venues': [{'id': '51bf3866498e55ee55df8db0',
    'name': 'The Fresh Italian',
    'location': {'lat': 43.654991437465284,
     'lng': -79.38789662900189,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.654991437465284,
       'lng': -79.38789662900189}],
     'distance': 127,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1568064125',
    'hasPerk': False},
   {'id': '573df789498e03dd8e54b166',
    'name': 'Mustachio Italian Eatery',
    'location': {'address': '595 Bay St',
     'crossStreet'

Write function to get all venues within 500 meters of postcode

In [349]:
def getNearbyVenues(names, latitudes, longitudes,  LIMIT, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #result2 = results["response"]['venues']
        
              # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

Get max 100 venues with 500 meters of neighbood

In [350]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                 latitudes=df2['Latitude'],
                                 longitudes=df2['Longitude'],
                                 LIMIT = 100
                                 )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [351]:
print(toronto_venues.shape)
toronto_venues.head()

(1294, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [352]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


Split the caterogies into Columns using one hot encoding

In [353]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [354]:
toronto_onehot.shape

(1294, 208)

In [355]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(18, 208)

Group venues and list them in descending order and display the top 10

In [356]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4       Cosmetics Shop  0.03
5           Restaurant  0.03
6                Hotel  0.03
7         Burger Joint  0.03
8  American Restaurant  0.03
9      Thai Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04
5      Farmers Market  0.04
6                Café  0.04
7          Steakhouse  0.04
8              Bakery  0.04
9         Fish Market  0.02


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2     Boat or Ferry  0.07
3          Boutique  0.07
4  Sculpture Garden  0.07
5               Bar  0.07
6   Harbor / Marina

The Canadians sure like their coffee shops

Function to find the most common venues and list the in descending order

In [357]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Find the top ten most common venues per neighborhood

In [358]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted 

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Cosmetics Shop,Burger Joint,Thai Restaurant,American Restaurant,Hotel,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Steakhouse,Seafood Restaurant,Beer Bar,Café,Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Coffee Shop,Boat or Ferry,Bar,Boutique,Sculpture Garden,Airport Lounge,Harbor / Marina,Airport Food Court
3,"Cabbagetown, St. James Town",Coffee Shop,Market,Italian Restaurant,Bakery,Restaurant,Café,Pub,Pizza Place,Gastropub,Butcher
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Spa,Indian Restaurant,Japanese Restaurant
5,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Bakery,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Caribbean Restaurant
6,Christie,Grocery Store,Café,Park,Baby Store,Restaurant,Italian Restaurant,Diner,Nightclub,Convenience Store,Coffee Shop
7,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Gastropub,Hotel,Fast Food Restaurant,Mediterranean Restaurant
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Seafood Restaurant,Steakhouse,Italian Restaurant,Deli / Bodega
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bakery,Deli / Bodega,American Restaurant,Italian Restaurant,Bar,Gastropub


Cluster analysis on the neighborhoods of Downtown Toronto

In [359]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([5, 5, 3, 5, 1, 5, 0, 5, 5, 5, 5, 4, 1, 1, 2, 5, 5, 5])

Make a new dataframe merging the origal dataframe with the cluster labels

In [360]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Trail,Playground,Building,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,5,Coffee Shop,Market,Italian Restaurant,Bakery,Restaurant,Café,Pub,Pizza Place,Gastropub,Butcher
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,5,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Gastropub,Hotel,Fast Food Restaurant,Mediterranean Restaurant
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Pub,Park,Bakery,Café,Gym / Fitness Center,Theater,Mexican Restaurant,Restaurant,Breakfast Spot
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,5,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Bookstore,Ramen Restaurant,Restaurant,Bakery,Plaza
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,5,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Breakfast Spot,Gastropub,Clothing Store,Cocktail Bar,Beer Bar
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,5,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Steakhouse,Seafood Restaurant,Beer Bar,Café,Restaurant
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Spa,Indian Restaurant,Japanese Restaurant
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,5,Coffee Shop,Café,Bar,Steakhouse,Cosmetics Shop,Burger Joint,Thai Restaurant,American Restaurant,Hotel,Restaurant
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,1,Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Baseball Stadium,Music Venue,History Museum


In [361]:
toronto_merged = toronto_merged.dropna(axis=0)
toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Trail,Playground,Building,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,5,Coffee Shop,Market,Italian Restaurant,Bakery,Restaurant,Café,Pub,Pizza Place,Gastropub,Butcher
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,5,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Gastropub,Hotel,Fast Food Restaurant,Mediterranean Restaurant
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Pub,Park,Bakery,Café,Gym / Fitness Center,Theater,Mexican Restaurant,Restaurant,Breakfast Spot
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,5,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Bookstore,Ramen Restaurant,Restaurant,Bakery,Plaza
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,5,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Breakfast Spot,Gastropub,Clothing Store,Cocktail Bar,Beer Bar
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,5,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Steakhouse,Seafood Restaurant,Beer Bar,Café,Restaurant
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Spa,Indian Restaurant,Japanese Restaurant
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,5,Coffee Shop,Café,Bar,Steakhouse,Cosmetics Shop,Burger Joint,Thai Restaurant,American Restaurant,Hotel,Restaurant
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,1,Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Baseball Stadium,Music Venue,History Museum


In [362]:
toronto_merged.groupby('Cluster Labels').describe()

Latitude                                                        \
                  count       mean       std        min        25%        50%   
Cluster Labels                                                                  
0                   1.0  43.669542       NaN  43.669542  43.669542  43.669542   
1                   3.0  43.651009  0.009019  43.640816  43.647538  43.654260   
2                   1.0  43.679563       NaN  43.679563  43.679563  43.679563   
3                   1.0  43.628947       NaN  43.628947  43.628947  43.628947   
4                   1.0  43.662696       NaN  43.662696  43.662696  43.662696   
5                  11.0  43.652843  0.007767  43.644771  43.647688  43.650571   

                                     Longitude                       \
                      75%        max     count       mean       std   
Cluster Labels                                                        
0               43.669542  43.669542       1.0 -79.422564       NaN   
1               43.656106  43.657952       3.0 -79.376590  0.014101   
2               43.679563  43.679563       1.0 -79.377529       NaN   
3               43.628947  43.628947       1.0 -79.394420       NaN   
4               43.662696  43.662696       1.0 -79.400049       NaN   
5               43.655184  43.667967      11.0 -79.380148  0.008281   

                                                                       
                      min        25%        50%        75%        max  
Cluster Labels                                                         
0              -79.422564 -79.422564 -79.422564 -79.422564 -79.422564  
1              -79.387383 -79.384567 -79.381752 -79.371194 -79.360636  
2              -79.377529 -79.377529 -79.377529 -79.377529 -79.377529  
3              -79.394420 -79.394420 -79.394420 -79.394420 -79.394420  
4              -79.400049 -79.400049 -79.400049 -79.400049 -79.400049  
5              -79.400049 -79.382720 -79.379817 -79.375132 -79.367675

Create a Map of the clusters

In [363]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [364]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Restaurant,Italian Restaurant,Diner,Nightclub,Convenience Store,Coffee Shop


Cluster 2

In [365]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Downtown Toronto,1,Coffee Shop,Pub,Park,Bakery,Café,Gym / Fitness Center,Theater,Mexican Restaurant,Restaurant,Breakfast Spot
57,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Spa,Indian Restaurant,Japanese Restaurant
59,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Baseball Stadium,Music Venue,History Museum


Cluster 3

In [366]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2,Park,Trail,Playground,Building,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 4

In [367]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,3,Airport Service,Airport Terminal,Coffee Shop,Boat or Ferry,Bar,Boutique,Sculpture Garden,Airport Lounge,Harbor / Marina,Airport Food Court


Cluster 5

In [368]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,4,Café,Theater,Japanese Restaurant,Sandwich Place,Bookstore,Restaurant,Bakery,Bar,Dessert Shop,Chinese Restaurant
